In [1]:
import os
import time

import samlab.dashboard

from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support.expected_conditions import presence_of_element_located as element_located

dashboard = samlab.dashboard.Server(config=False, quiet=True)
dashboard.ready(timeout=10)


options = Options()
options.headless = True
options.add_argument("--width=1024")
options.add_argument("--height=768")

browser = Firefox(options=options)
browser.get(dashboard.uri)
Wait(browser, timeout=10).until(element_located((By.ID, "samlab-notify-container")))

os.makedirs("dashboard", exist_ok=True)

print(dashboard.uri)

http://127.0.0.1:4000


`$ samlab-dashboard`

The server will start, and automatically display the dashboard interface in a web browser:

In [2]:
time.sleep(1.0)
browser.save_screenshot(os.path.join(os.getcwd(), "dashboard/welcome.png"))
time.sleep(1.0)

<img class="browser" src="dashboard/welcome.png">

In [3]:
browser.find_element_by_xpath('//button[normalize-space()="Operations"]').click()
time.sleep(1.0)
browser.save_screenshot(os.path.join(os.getcwd(), "dashboard/operations.png"))
time.sleep(1.0)

browser.find_element_by_xpath('//button[normalize-space()="Operations"]').click()
browser.find_element_by_xpath('//button[normalize-space()="Datasets"]').click()
time.sleep(1.0)
browser.save_screenshot(os.path.join(os.getcwd(), "dashboard/empty-datasets.png"))
time.sleep(1.0)

browser.find_element_by_xpath('//button[normalize-space()="Datasets"]').click()
browser.find_element_by_xpath('//button[normalize-space()="Favorites"]').click()
time.sleep(1.0)
browser.save_screenshot(os.path.join(os.getcwd(), "dashboard/empty-favorites.png"))
time.sleep(1.0)

browser.find_element_by_xpath('//button[normalize-space()="Favorites"]').click()

Let's examine the menus from right-to-left.  The Operations menu provides a set of generic commands for managing the dashboard layout, information lists, and displaying acknowlegements:

<img class="browser" src="dashboard/operations.png">

The Datasets menu lists all of the data sources that have been registered with the server. Since we haven't registered any yet, the menu is empty: 

<img class="browser" src="dashboard/empty-datasets.png">

The Favorites menu lists dashboard layouts that have been favorited for later recall or sharing with collaborators.  Once again, the list is empty, since we haven't favorited anything yet:

<img class="browser" src="dashboard/empty-favorites.png">

In [4]:
browser.close()
dashboard.stop()